##Librerie

In [ ]:
###LIBRERIE 
import numpy as np 
import pandas as pd 
import os
import keras
import matplotlib.pyplot as plt
import tensorflow as tf

from google.colab import drive
from tensorflow.python.client import device_lib
from PIL import Image
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from keras import layers, models
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras import backend




# Inserimento dei dati


In [ ]:
###DATA PREPROCESSING 
drive.mount('/content/drive', force_remount = True)

folder_benign_train = '/content/drive/MyDrive/Colab/train/benign'
folder_malignant_train = '/content/drive/MyDrive/Colab/train/malignant'

folder_benign_test = '/content/drive/MyDrive/Colab/test/benign'
folder_malignant_test = '/content/drive/MyDrive/Colab/test/malignant'

read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))

# Carico le immagini di train
ims_benign = [read(os.path.join(folder_benign_train, filename)) for filename in os.listdir(folder_benign_train)]
X_benign = np.array(ims_benign, dtype='uint8')
print("train beningi caricati")
ims_malignant = [read(os.path.join(folder_malignant_train, filename)) for filename in os.listdir(folder_malignant_train)]
X_malignant = np.array(ims_malignant, dtype='uint8')
print("train maligni caricati")

# Carico le immagini di test
ims_benign = [read(os.path.join(folder_benign_test, filename)) for filename in os.listdir(folder_benign_test)]
X_benign_test = np.array(ims_benign, dtype='uint8')
print("test beningi caricati")
ims_malignant = [read(os.path.join(folder_malignant_test, filename)) for filename in os.listdir(folder_malignant_test)]
X_malignant_test = np.array(ims_malignant, dtype='uint8')
print("test maligni caricati")

##Creo le Lables


In [ ]:

y_benign = np.zeros(X_benign.shape[0])
y_malignant = np.ones(X_malignant.shape[0])

y_benign_test = np.zeros(X_benign_test.shape[0])
y_malignant_test = np.ones(X_malignant_test.shape[0])

##Merge and Shuffle


In [ ]:
X_train = np.concatenate((X_benign, X_malignant), axis = 0)
y_train = np.concatenate((y_benign, y_malignant), axis = 0)

X_test = np.concatenate((X_benign_test, X_malignant_test), axis = 0)
y_test = np.concatenate((y_benign_test, y_malignant_test), axis = 0)


#Shuffle
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
y_train = y_train[s]

s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
y_test = y_test[s]

##Visualizzo le immagini

In [ ]:
fig=plt.figure(figsize=(17, 8))
columns = 8
rows = 5

for i in range(1, columns*rows +1):
    ax = fig.add_subplot(rows, columns, i)
    if y_train[i] == 0:
        ax.title.set_text('Benign')
    else:
        ax.title.set_text('Malignant')
    plt.imshow(X_train[i], interpolation='nearest')
plt.show()

##Lables-->Categorical

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
y_test =tf.keras.utils.to_categorical(y_test, num_classes=2)

##Normalizzazione

In [ ]:
X_train = X_train/255
X_test = X_test/255
X_train.shape

##Modello

In [ ]:
#in optin da provare con rmsprop , adam e in activation_Dense con sigmoid , softmax
def Addestramento(input_shape, lr, numclassi, optim, activation_Dense): 
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), padding = 'same',activation = 'relu', input_shape = input_shape,kernel_initializer='glorot_uniform'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.40))#metto questo stato di dropout per evitare un overfitting della rete 
    model.add(layers.Conv2D(64, (3,3), padding = 'same' ,activation = 'relu', kernel_initializer='glorot_uniform'))
    model.add(MaxPool2D(2,2))
    model.add(Dropout(0.40))
    model.add(layers.Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(numclassi, activation='softmax'))
    #Scegli l'ottimizzatore
    if optim == 'rmsprop':
        optimizer = RMSprop(lr=lr)
    else:
        optimizer = Adam(lr=lr)
    model.summary()

    model.compile(optimizer = optimizer , loss = "binary_crossentropy", metrics=["accuracy"])
    return model


##Addestramento

In [ ]:
model = Addestramento((224,224,3), 1e-5, 2,'relu', 'adam')

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=5,  verbose=1, factor=0.5,  min_lr=1e-7)

epochs = 1
batch_size = 64

history = model.fit(X_train, y_train, validation_split=0.2, epochs= epochs , batch_size= batch_size,callbacks=[learning_rate_reduction], verbose=1)
#Lista di tutta la storia fatta dai dati 
print(history.history.keys())
# storia accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#storia loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')

In [ ]:
backend.clear_session()
del model
del history

##K-fold


In [ ]:
# Merge inputs and targets
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

#3 k fold
kfold = KFold(n_splits=3, shuffle=True)
print(f'values kfold = {kfold}')

#score container
acc_per_fold = []
loss_per_fold = []
fold_no = 1
for train, test in kfold.split(inputs, targets):

    model = Addestramento((224,224,3), 1e-5, 2,'relu', 'adam')
    print("rete caricata")
    
    #Addestro il modello su una parte dei dati di train
    model.fit(X_train[train], y_train[train], epochs=epochs, batch_size=batch_size, verbose=1)
    print('fine addestramento rete')

    # provo il modello sui dati di test rimanenti
    scores = model.evaluate(X_train[test], y_train[test], verbose=1)
    print(f'score ottenuto dalla rete{scores}')
    
    #score del modello in percentuale
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score per fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    del model

#Media degli score
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Media degli scores per tutte le folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')